# Simulation experiment 

Run entire simulation experiment multiple times to generate confidence interval

In [1]:
%load_ext autoreload
%autoreload 2

from joblib import Parallel, delayed
import multiprocessing
import sys
import os
import pandas as pd

import warnings
warnings.filterwarnings(action='ignore')

sys.path.append("../")
from functions import pipelines

from numpy.random import seed
randomState = 123
seed(randomState)

Using TensorFlow backend.


In [ ]:
# Parameters
NN_architecture = 'NN_2500_30'
analysis_name = 'analysis_0'
num_simulated_samples = 6000
lst_num_experiments = [1, 2, 5, 10, 20,
                     50, 100, 500, 1000, 2000, 3000, 6000]
corrected = False
use_pca = True
num_PCs = 10

iterations = range(10) 
num_cores = 5

In [ ]:
# Input file
base_dir = os.path.abspath(
      os.path.join(
          os.getcwd(), "../.."))

normalized_data_file = os.path.join(
      base_dir,
      "data",
      "input",
      "train_set_normalized.pcl")

In [2]:
# Output files
local_dir = "/home/alexandra/Documents/"

similarity_uncorrected_file = os.path.join(
    local_dir,
    "Data",
    "Batch_effects",
    "output",
    "analysis_0_similarity_uncorrected.pickle")

ci_uncorrected_file = os.path.join(
    local_dir,
    "Data",
    "Batch_effects",
    "output",
    "analysis_0_ci_uncorrected.pickle")

In [3]:
# Run multiple simulations - uncorrected

results = Parallel(n_jobs=num_cores, verbose=100)(
    delayed(
        pipelines.simple_simulation_experiment_uncorrected)(i,
                                                            NN_architecture,
                                                            analysis_name,
                                                            num_simulated_samples,
                                                            lst_num_experiments,
                                                            corrected,
                                                            use_pca,
                                                            num_PCs,
                                                            "Experiment",
                                                            normalized_data_file) for i in iterations)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   1 tasks      | elapsed:  3.8min
[Parallel(n_jobs=5)]: Done   2 out of  10 | elapsed:  3.9min remaining: 15.5min
[Parallel(n_jobs=5)]: Done   3 out of  10 | elapsed:  3.9min remaining:  9.1min
[Parallel(n_jobs=5)]: Done   4 out of  10 | elapsed:  4.0min remaining:  6.0min
[Parallel(n_jobs=5)]: Done   5 out of  10 | elapsed:  4.0min remaining:  4.0min
[Parallel(n_jobs=5)]: Done   6 out of  10 | elapsed:  7.5min remaining:  5.0min
[Parallel(n_jobs=5)]: Done   7 out of  10 | elapsed:  7.6min remaining:  3.3min
[Parallel(n_jobs=5)]: Done   8 out of  10 | elapsed:  7.7min remaining:  1.9min
[Parallel(n_jobs=5)]: Done  10 out of  10 | elapsed:  7.8min remaining:    0.0s
[Parallel(n_jobs=5)]: Done  10 out of  10 | elapsed:  7.8min finished


In [4]:
# Concatenate output dataframes
all_svcca_scores = pd.DataFrame()

for i in iterations:
    all_svcca_scores = pd.concat([all_svcca_scores, results[i][1]], axis=1)

all_svcca_scores

,score,score,score,score,score,score,score,score,score,score
number of experiments,,,,,,,,,,
1,0.999994,0.999994,0.999994,0.999994,0.999994,0.999994,0.999994,0.999994,0.999994,0.999994
2,0.901761,0.900751,0.902341,0.901195,0.901537,0.901960,0.900591,0.901738,0.901833,0.900161
5,0.607209,0.608213,0.606863,0.611047,0.610162,0.609071,0.607958,0.609651,0.609955,0.607965
10,0.130080,0.133383,0.131133,0.132351,0.133170,0.131531,0.127556,0.128726,0.136425,0.125894
20,0.048666,0.046363,0.039898,0.046762,0.047476,0.043265,0.046188,0.038261,0.040468,0.040145
50,0.152640,0.151998,0.144800,0.153213,0.144034,0.151680,0.146229,0.148812,0.144961,0.156377
100,0.228591,0.235202,0.232042,0.233949,0.238646,0.236454,0.234850,0.242122,0.239245,0.238676
500,0.697937,0.704116,0.743546,0.726046,0.715950,0.718000,0.741065,0.701284,0.734960,0.733642
1000,0.939947,0.943675,0.937064,0.934701,0.939832,0.933130,0.941193,0.936527,0.940996,0.937253


In [5]:
# Get median for each row (number of experiments)
mean_scores = all_svcca_scores.mean(axis=1).to_frame()
mean_scores.columns = ['score']
mean_scores

,score
number of experiments,
1,0.999994
2,0.901387
5,0.608809
10,0.131025
20,0.043749
50,0.149474
100,0.235978
500,0.721654
1000,0.938432


In [6]:
# Get standard dev for each row (number of experiments)
import math
std_scores = (all_svcca_scores.std(axis=1)/math.sqrt(10)).to_frame()
std_scores.columns = ['score']
std_scores

,score
number of experiments,
1,3.635618e-08
2,2.185911e-04
5,4.355264e-04
10,9.769144e-04
20,1.196621e-03
50,1.356729e-03
100,1.243188e-03
500,5.298866e-03
1000,1.026616e-03


In [7]:
# Get confidence interval for each row (number of experiments)
# z-score for 95% confidence interval 
err = std_scores*1.96

In [8]:
# Get boundaries of confidence interval
ymax = mean_scores + err
ymin = mean_scores - err

ci = pd.concat([ymin, ymax], axis=1)
ci.columns = ['ymin', 'ymax']
ci

,ymin,ymax
number of experiments,,
1,0.999994,0.999994
2,0.900958,0.901815
5,0.607956,0.609663
10,0.129110,0.132940
20,0.041404,0.046094
50,0.146815,0.152134
100,0.233541,0.238414
500,0.711269,0.732040
1000,0.936420,0.940444


In [9]:
mean_scores

,score
number of experiments,
1,0.999994
2,0.901387
5,0.608809
10,0.131025
20,0.043749
50,0.149474
100,0.235978
500,0.721654
1000,0.938432


In [10]:
# Pickle dataframe of mean scores scores for first run, interval
mean_scores.to_pickle(similarity_uncorrected_file)
ci.to_pickle(ci_uncorrected_file)